In [1]:
!python --version

Python 3.11.10


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#import random
import numpy as np
import pandas as pd

In [4]:
from line_profiler import LineProfiler

In [5]:
import matplotlib.pyplot as plt

In [6]:
%matplotlib inline

In [231]:
np.random.rand()

0.37105590256799303

In [232]:
class World():
    """Class World - Макромир, который задает начало отсчета времени,
        законы макроэкономики, ограничения регуляторов/ЦБ и остальное окружение.
        Так же здесь описывается поведение клиента, которое может зависеть от внешних факторов.

        dod_migration - матрица вероятностей переходов по просрочкам
    """

    def __init__(self):
        self.World_Time = 0
        #                             0    1+   31+   61+   91+   WOF
        self.dod_migration = np.array([[0.95, 0.05, 0.00, 0.00, 0.00, 0.00], #  0 
                                       [0.90, 0.05, 0.05, 0.00, 0.00, 0.00], #  1+
                                       [0.10, 0.05, 0.05, 0.80, 0.00, 0.00], # 31+
                                       [0.05, 0.05, 0.05, 0.05, 0.80, 0.00], # 61+
                                       [0.01, 0.01, 0.02, 0.02, 0.04, 0.90], # 91+
                                       [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # WOF
            #                          [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # TODO - добавить досрочное и частичнодочсрочное погашение кредита 
                         ])
        assert self.dod_migration.shape[0] == self.dod_migration.shape[1] # проверка на квадратность
        assert [i.sum() for i in self.dod_migration] == [1 for i in range(len(self.dod_migration))] # 1 in sum of row
        print('test - ', [i.sum() for i in self.dod_migration])

    def print_current_reality(self):
        print('Welcome to the real world')

    # возвращаем предопределенный скор, который неизменен
    def get_god_score(self):
        return np.random.rand()

In [5]:
class DWH_DB():
    """Class DWH - база данных
    """
    def __init__(self):
        self.LI = pd.DataFrame(columns = ['CNTR_ID',
                                          'SD',
                                          'DOD_ID',
                                          'MOB',
                                          'WRTOFF_ID',
                                          'CLOSED_ID'
                                          ])
        self.DMContract = pd.DataFrame(columns = ['CNTR_ID',
                                                  'ISSUE_DT',
                                                  'WRTOFF_DT',
                                                  'CLOSED_DT'
                                                 ])
        


In [151]:
class Tariff():
    """Class Tariff - здесь опишем тарифы.
    """

    def __init__(self,name, IR = 0.12, DUR = 24, TypePlan = 'Annuity'):
        self.IR = IR              # годовая процентная ставка
        self.DUR = DUR            # срок
        self.TypePlan = TypePlan  # тип кредита
        self.PlanParam = {}       # справочник, для описания различных типов/подходов кредита  
        self.plan()               # заполним график платежей

    # создание графика платежей
    def plan(self):
        PP = self.PlanParam
        if self.TypePlan == 'Annuity':
            i = self.IR/12 # Ставка в месяц
            n = self.DUR   # Срок в месяцах
            d = (1+i)**n   # вспомогательная переменная
            K = i*d/(d-1)  # Коэффициент аннуитета
            PP['K'] = K    # Коэффициент аннуитета
            PP['MD'] =  1/i- n/(d-1)/(1+i) # Модифицированная дюрация
            # опишем график платежей: 
            PP[1] = np.array([1+i, # Долг на момент оплаты
                              K,   # платеж
                              K-i, # долг (часть платежа)
                              i,   # проценты (часть платежа)
                              K-i, # кумулятив по оплаченному долгу
                              i,   # кумулятив по оплаченным процентам
                              K,   # кумулятив по оплатам
                             ])
            for j in range(2, n+1):
                last_EAD = (PP[j-1][0]-K) 
                PP[j] = np.array([last_EAD*(1+i), K , K-last_EAD*i, last_EAD*i, K-last_EAD*i + PP[j-1][4], last_EAD*i + PP[j-1][5], K*j])

    # Краткая инфа о тарифном плане
    def info(self):
        if self.TypePlan == 'Annuity':
            print('TypePlan = %s\n IR = %s\n Duration = %i\n MD = %g'%(self.TypePlan, self.IR, self.DUR, self.PlanParam['MD']))

    # Нарисовать график для заданной суммы
    def print_plan(self, amount = 1):
        if self.TypePlan == 'Annuity':
            print(' N', ['%10s'%c for c in ['Долг+проц', 'Платеж', 'Плат(долг)', 'Плат(проц)', 'Кум долг', 'Кум проц', 'Кум оплаты']])
            for i in range(1, self.DUR+1):
                print('%02i'%i, ['%10s'%('%0.2f'%(j*amount)) for j in self.PlanParam[i]])


In [152]:
tariff_01 = Tariff('TR01')
tariff_01.info()

TypePlan = Annuity
 IR = 0.12
 Duration = 24
 MD = 11.9046


In [153]:
tariff_02 = Tariff('TR02', IR = 0.18, DUR = 36)
tariff_02.info()

TypePlan = Annuity
 IR = 0.18
 Duration = 36
 MD = 16.6512


In [154]:
tariff_01.print_plan(100_000)

 N [' Долг+проц', '    Платеж', 'Плат(долг)', 'Плат(проц)', '  Кум долг', '  Кум проц', 'Кум оплаты']
01 [' 101000.00', '   4707.35', '   3707.35', '   1000.00', '   3707.35', '   1000.00', '   4707.35']
02 ['  97255.58', '   4707.35', '   3744.42', '    962.93', '   7451.77', '   1962.93', '   9414.69']
03 ['  93473.71', '   4707.35', '   3781.86', '    925.48', '  11233.63', '   2888.41', '  14122.04']
04 ['  89654.03', '   4707.35', '   3819.68', '    887.66', '  15053.32', '   3776.07', '  18829.39']
05 ['  85796.15', '   4707.35', '   3857.88', '    849.47', '  18911.20', '   4625.54', '  23536.74']
06 ['  81899.69', '   4707.35', '   3896.46', '    810.89', '  22807.66', '   5436.43', '  28244.08']
07 ['  77964.27', '   4707.35', '   3935.42', '    771.92', '  26743.08', '   6208.35', '  32951.43']
08 ['  73989.49', '   4707.35', '   3974.78', '    732.57', '  30717.86', '   6940.92', '  37658.78']
09 ['  69974.96', '   4707.35', '   4014.53', '    692.82', '  34732.38', '   7633

In [155]:
tariff_02.print_plan(100_000)

 N [' Долг+проц', '    Платеж', 'Плат(долг)', 'Плат(проц)', '  Кум долг', '  Кум проц', 'Кум оплаты']
01 [' 101500.00', '   3615.24', '   2115.24', '   1500.00', '   2115.24', '   1500.00', '   3615.24']
02 ['  99353.03', '   3615.24', '   2146.97', '   1468.27', '   4262.21', '   2968.27', '   7230.48']
03 ['  97173.86', '   3615.24', '   2179.17', '   1436.07', '   6441.38', '   4404.34', '  10845.72']
04 ['  94962.00', '   3615.24', '   2211.86', '   1403.38', '   8653.24', '   5807.72', '  14460.96']
05 ['  92716.96', '   3615.24', '   2245.04', '   1370.20', '  10898.28', '   7177.92', '  18076.20']
06 ['  90438.25', '   3615.24', '   2278.71', '   1336.53', '  13176.99', '   8514.44', '  21691.44']
07 ['  88125.35', '   3615.24', '   2312.89', '   1302.35', '  15489.89', '   9816.79', '  25306.68']
08 ['  85777.76', '   3615.24', '   2347.59', '   1267.65', '  17837.47', '  11084.44', '  28921.92']
09 ['  83394.96', '   3615.24', '   2382.80', '   1232.44', '  20220.28', '  12316

In [42]:
class Contract():
    """Class Contract
       issue_dt - issue of contract
       duration - duration in months
    """

    dod_dic = {0: '0',
               1: '1+',
               2: '31+',
               3: '61+',
               4: '91+',
               5: 'WOF'
              }
    dod_cnt = 6 # кол-во состояний
    dod_states = np.eye(dod_cnt) # матрица состояний (для удобства использована единичная матрица)

    def __init__(self, cntr_id = 0, issue_dt = 0, duration = 0,
                 world = World, tariff = Tariff, amount = 100_000):
        self.cntr_id = cntr_id
        self.dod_id = 0        # начальное состояние контракта при выдачи: DOD = 0
        self.dod_state = self.dod_states[0] # np.array([1,0,0,0,0]) 
        self.dod_migration = world.dod_migration
        self.issue_dt = issue_dt
        self.mob = 0
        self.duration = duration
        self.closed_id = 0       # 0 - контратк открыт, 1 - закрыт
        self.wrtoff_id = 0       # 0 - контратк несписан, 1 - списан
        
    def next_month(self):
        if self.closed_id == 1:
            return None
           
        self.mob = self.mob + 1
        p = self.dod_migration[self.dod_id]                    # array of probabilities
        self.dod_id = np.random.choice(self.dod_cnt,1,p=p)[0]  # new state
        self.dod_state = self.dod_states[self.dod_id]

        if self.dod_id == 0 and self.mob >= self.duration: # погашение либо выздоровление с возвращением в график
            self.closed_id = 1
        
        if self.dod_id == 5 and self.mob >= self.duration + 12: # списание
            self.wrtoff_id = 1

        if self.wrtoff_id == 1 and self.mob >= self.duration + 24: # закрытие списанного контракта
            self.closed_id = 1
            

In [70]:
class Portfolio():
    """Class Portfolio - Портфель - динамика 
        N - первая выдача при создании портфеля
        start_portfolio_dt - привязка портфеля к мировому времени - важно при наличии нескольких портфелей
    
    """
    def __init__(self, N = 10, duration = 36, start_portfolio_dt = 0, world = None):
        self.cntr_id = 0                                # счетчик контрактов
        self.start_portfolio_dt = start_portfolio_dt    # дата создания портфеля
        self.cntr_list = []                             # сам портфель - список контрактов
        self.portfolio_age = 0                          # возрвст портфеля
        self.world = world

        # проведем первую выдачу - инициализация портфеля
#        self.issue(N, duration)
        # Заполним LI
#        self.fix_in_dwh()

    def issue(self, issue_plan = [(Tariff,0)]):
        for i in range(len(issue_plan)):
            self.cntr_id += 1
            cntr_tariff = issue_plan[i][0]
            cntr_amount = issue_plan[i][1]
            self.cntr_list.append(Contract(cntr_id = self.cntr_id, 
                                           issue_dt = self.start_portfolio_dt,
                                           duration = cntr_tariff.DUR,
                                           world = self.world,
                                           tariff = cntr_tariff,
                                           amount = cntr_amount
                                          ))

    def next_month(self, issue_plan = [(Tariff,0)], log = False):
        self.portfolio_age +=1

        # Для проверки - выведем все закрытые на этот момент контракты
        if log:
            test = [cntr.cntr_id for cntr in self.cntr_list if cntr.closed_id == 1 ]
            print('%04i' % self.portfolio_age, len(self.cntr_list), 'out ->',  test)            

        # Перезапишем список только открытыми контрактами 
        self.cntr_list = [cntr for cntr in self.cntr_list if cntr.closed_id == 0 ]
        
        # сдвинем существующий портфель, потом проведем выдачу новых         
        for cntr in self.cntr_list:
            cntr.next_month()
            
        # проведем выдачи
        self.issue(issue_plan)

        # Заполним LI
        self.fix_in_dwh()


    def fix_in_dwh_old(self): # Пример медленной вставки
        ix = len(DWH.LI.index)
        for cnt in self.cntr_list:
            DWH.LI.loc[ix] = [cnt.cntr_id, self.portfolio_age, cnt.dod_id, cnt.mob]
            ix += 1

    def fix_in_dwh(self):
        fix_data = [[cnt.cntr_id, self.portfolio_age, cnt.dod_id, cnt.mob, cnt.wrtoff_id, cnt.closed_id] for cnt in self.cntr_list]
        DWH.LI = pd.concat([DWH.LI,
                            pd.DataFrame(data=fix_data,
                                         columns=DWH.LI.columns)
                           ])


In [167]:
%time
N_const = 1000 # Пусть будут постоянные ежемесячные выдачи 
duration = 24
WW = World()
DWH = DWH_DB()
GP = Portfolio(N_const, duration, world = WW)
%time

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 8.58 μs
test -  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.96 μs


In [177]:
GP.portfolio_age

30

In [170]:
issue_plan = [(tariff_01, 100_000) for i in range(N_const//2)] + [(tariff_02, 100_000) for i in range(N_const//2)]

In [173]:
#print([i[0].DUR for i in issue_plan])

In [178]:
%time
for t in range(100):
    GP.next_month(issue_plan)

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 8.11 μs


In [179]:
WW.dod_migration

array([[0.95, 0.05, 0.  , 0.  , 0.  , 0.  ],
       [0.9 , 0.05, 0.05, 0.  , 0.  , 0.  ],
       [0.1 , 0.05, 0.05, 0.8 , 0.  , 0.  ],
       [0.05, 0.05, 0.05, 0.05, 0.8 , 0.  ],
       [0.01, 0.01, 0.02, 0.02, 0.04, 0.9 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

In [180]:
T = DWH.LI.reset_index(drop=True)
print(T.shape)
T.tail()

(3661158, 6)


,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID
3661153,129996,130,0,0,0,0
3661154,129997,130,0,0,0,0
3661155,129998,130,0,0,0,0
3661156,129999,130,0,0,0,0
3661157,130000,130,0,0,0,0


In [182]:
T[T['CNTR_ID']==700]

,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID
699,700,1,0,0,0,0
1699,700,2,0,1,0,0
3699,700,3,0,2,0,0
6699,700,4,0,3,0,0
10699,700,5,0,4,0,0
15699,700,6,0,5,0,0
21699,700,7,0,6,0,0
28699,700,8,0,7,0,0
36699,700,9,0,8,0,0
45699,700,10,0,9,0,0


In [183]:
G = T.groupby('SD')[['MOB']].agg(['count','mean']).reset_index()
G.columns = ['SD','CNT','MEAN']
G

,SD,CNT,MEAN
0,1,1000,0.0
1,2,2000,0.5
2,3,3000,1.0
3,4,4000,1.5
4,5,5000,2.0
...,...,...,...
125,126,32303,16.686964
126,127,32304,16.686602
127,128,32298,16.678246
128,129,32327,16.698859


In [184]:
G.groupby('CNT')['SD'].agg(['count','min']).tail(20)

,count,min
CNT,,
32332,2,86
32333,1,78
32334,1,71
32335,1,108
32336,1,75
32337,1,87
32340,1,88
32342,1,91
32344,1,100


In [185]:
m_lag = 12  # период переката

ix_sd   = T['SD'] >= 40 # для кредитов сроком 24 мес
#ix_dod  = T['DOD_ID'] < 4 # не дефолты
ix_open = T['CLOSED_ID'] == 0 # не закрытые
ix_mlag = T['SD'] <= (T['SD'].max()-m_lag) # ограничение на вызревание
D_F = T.loc[ix_sd & ix_mlag & ix_open, :]

In [186]:
D_F

,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID
728621,23,40,5,39,1,0
728622,25,40,5,39,1,0
728623,26,40,5,39,1,0
728624,40,40,5,39,1,0
728625,57,40,5,39,1,0
...,...,...,...,...,...,...
3273589,117996,118,0,0,0,0
3273590,117997,118,0,0,0,0
3273591,117998,118,0,0,0,0
3273592,117999,118,0,0,0,0


In [187]:
D_N = T.copy()
D_N['SD_OLD'] = D_N['SD']
D_N['SD'] = D_N['SD']-m_lag

In [188]:
D_N

,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID,SD_OLD
0,1,-11,0,0,0,0,1
1,2,-11,0,0,0,0,1
2,3,-11,0,0,0,0,1
3,4,-11,0,0,0,0,1
4,5,-11,0,0,0,0,1
...,...,...,...,...,...,...,...
3661153,129996,118,0,0,0,0,130
3661154,129997,118,0,0,0,0,130
3661155,129998,118,0,0,0,0,130
3661156,129999,118,0,0,0,0,130


In [189]:
D = D_F.merge(D_N, on = ['CNTR_ID','SD'], how = 'left', suffixes=['_F','_N'])
D.shape

(2466789, 11)

In [190]:
# развилка - оставлять или удалять такие записи
D[D['DOD_ID_N'].isna()]

,CNTR_ID,SD,DOD_ID_F,MOB_F,WRTOFF_ID_F,CLOSED_ID_F,DOD_ID_N,MOB_N,WRTOFF_ID_N,CLOSED_ID_N,SD_OLD
0,23,40,5,39,1,0,NaN,NaN,NaN,NaN,NaN
1,25,40,5,39,1,0,NaN,NaN,NaN,NaN,NaN
2,26,40,5,39,1,0,NaN,NaN,NaN,NaN,NaN
3,40,40,5,39,1,0,NaN,NaN,NaN,NaN,NaN
4,57,40,5,39,1,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2453284,104496,118,0,13,0,0,NaN,NaN,NaN,NaN,NaN
2453285,104497,118,0,13,0,0,NaN,NaN,NaN,NaN,NaN
2453286,104498,118,0,13,0,0,NaN,NaN,NaN,NaN,NaN
2453287,104499,118,0,13,0,0,NaN,NaN,NaN,NaN,NaN


In [191]:
D_v1 = D.copy()
D_v2 = D.copy()

In [192]:
# 1 - Удалим
# Это контракты, которые были погашены
ix = D_v1['DOD_ID_N'].isna()
D_v1 = D_v1[~ix]
D_v1.shape

# 2 - переопределим в просрочку в 0, если а прошлом периоде не было списания, иначе 5
ix_0 = (D_v2['DOD_ID_N'].isna()) & (D_v2['WRTOFF_ID_F'] == 0)
ix_5 = (D_v2['DOD_ID_N'].isna()) & (D_v2['WRTOFF_ID_F'] == 1)
D_v2.loc[ix_0,'DOD_ID_N'] = 0
D_v2.loc[ix_5,'DOD_ID_N'] = 5
D_v2.shape


(2466789, 11)

In [193]:
D_v1['DOD_F'] = D_v1['DOD_ID_F'].apply(lambda x: 'D_%s'%x)
D_v1['DOD_N'] = D_v1['DOD_ID_N'].apply(lambda x: 'D_%s'%x)

D_v2['DOD_F'] = D_v2['DOD_ID_F'].apply(lambda x: 'D_%s'%x)
D_v2['DOD_N'] = D_v2['DOD_ID_N'].apply(lambda x: 'D_%s'%x)

In [194]:
M_v1 = D_v1.fillna(-1).groupby([#'SD',
               'DOD_F','WRTOFF_ID_F','CLOSED_ID_F','DOD_N','WRTOFF_ID_N','CLOSED_ID_N']
             )['CNTR_ID'].count().reset_index()
M_v1

/tmp/ipykernel_120719/2508835775.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  M_v1 = D_v1.fillna(-1).groupby([#'SD',


,DOD_F,WRTOFF_ID_F,CLOSED_ID_F,DOD_N,WRTOFF_ID_N,CLOSED_ID_N,CNTR_ID
0,D_0,0,0,D_0,0,0,1238333
1,D_0,0,0,D_0,0,1,71080
2,D_0,0,0,D_1,0,0,69689
3,D_0,0,0,D_2,0,0,4176
4,D_0,0,0,D_3,0,0,3827
5,D_0,0,0,D_4,0,0,3299
6,D_0,0,0,D_5,0,0,27257
7,D_1,0,0,D_0,0,0,59618
8,D_1,0,0,D_0,0,1,3620
9,D_1,0,0,D_1,0,0,3473


In [195]:
M_v2 = D_v2.fillna(-1).groupby([#'SD',
               'DOD_F','WRTOFF_ID_F','CLOSED_ID_F','DOD_N','WRTOFF_ID_N','CLOSED_ID_N']
             )['CNTR_ID'].count().reset_index()
M_v2

/tmp/ipykernel_120719/1970697240.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  M_v2 = D_v2.fillna(-1).groupby([#'SD',


,DOD_F,WRTOFF_ID_F,CLOSED_ID_F,DOD_N,WRTOFF_ID_N,CLOSED_ID_N,CNTR_ID
0,D_0,0,0,D_0,-1,-1,777795
1,D_0,0,0,D_0,0,0,1238333
2,D_0,0,0,D_0,0,1,71080
3,D_0,0,0,D_1,0,0,69689
4,D_0,0,0,D_2,0,0,4176
5,D_0,0,0,D_3,0,0,3827
6,D_0,0,0,D_4,0,0,3299
7,D_0,0,0,D_5,0,0,27257
8,D_1,0,0,D_0,-1,-1,43929
9,D_1,0,0,D_0,0,0,59618


In [196]:
MM_v1 = M_v1.pivot_table(index='DOD_F', columns='DOD_N', values='CNTR_ID', aggfunc='sum', fill_value=0)
MM_v1['SUM'] = MM_v1.sum(axis=1)
MM_v1

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5,SUM
DOD_F,,,,,,,
D_0,1309413,69689,4176,3827,3299,27257,1417661
D_1,63238,3473,230,189,163,5507,72800
D_2,906,64,0,3,1,4706,5680
D_3,396,27,2,0,0,4777,5202
D_4,55,3,1,1,0,4530,4590
D_5,0,0,0,0,0,97832,97832


In [197]:
MM_v2 = M_v2.pivot_table(index='DOD_F', columns='DOD_N', values='CNTR_ID', aggfunc='sum', fill_value=0)
MM_v2['SUM'] = MM_v2.sum(axis=1)
MM_v2

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5,SUM
DOD_F,,,,,,,
D_0,2087208,69689,4176,3827,3299,27257,2195456
D_1,107167,3473,230,189,163,5507,116729
D_2,1735,64,0,3,1,4706,6509
D_3,808,27,2,0,0,4777,5614
D_4,120,3,1,1,0,4530,4655
D_5,0,0,0,0,0,137826,137826


In [198]:
for c in MM_v1.columns:
    MM_v1[c] = MM_v1[c]/MM_v1['SUM']
MM_v1.drop(columns='SUM', inplace=True)    
MM_v1 = MM_v1.astype(np.float16)

for c in MM_v2.columns:
    MM_v2[c] = MM_v2[c]/MM_v2['SUM']
MM_v2.drop(columns='SUM', inplace=True)    
MM_v2 = MM_v2.astype(np.float16)

In [199]:
MM_v1

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5
DOD_F,,,,,,
D_0,0.923828,0.049164,0.002945,0.002699,0.002327,0.019226
D_1,0.868652,0.047699,0.003159,0.002596,0.002239,0.075623
D_2,0.159546,0.011269,0.000000,0.000528,0.000176,0.828613
D_3,0.076111,0.005192,0.000385,0.000000,0.000000,0.918457
D_4,0.011986,0.000654,0.000218,0.000218,0.000000,0.986816
D_5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [200]:
# MM_06_v1 = MM_v1
# MM_06_v2 = MM_v2

In [201]:
# MM_12_v1 = MM_06_v1 @ MM_06_v1
# MM_12_v2 = MM_06_v2 @ MM_06_v2
# MM_12_v1

In [202]:
MM_12_v1 = MM_v1
MM_12_v2 = MM_v2
MM_12_v1

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5
DOD_F,,,,,,
D_0,0.923828,0.049164,0.002945,0.002699,0.002327,0.019226
D_1,0.868652,0.047699,0.003159,0.002596,0.002239,0.075623
D_2,0.159546,0.011269,0.000000,0.000528,0.000176,0.828613
D_3,0.076111,0.005192,0.000385,0.000000,0.000000,0.918457
D_4,0.011986,0.000654,0.000218,0.000218,0.000000,0.986816
D_5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [203]:
MM_12_v1 - MM_12_v2

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5
DOD_F,,,,,,
D_0,-0.026855,0.017426,0.001042,0.000956,0.000824,0.006813
D_1,-0.049316,0.017944,0.001188,0.000977,0.000843,0.028442
D_2,-0.107056,0.001434,0.000000,0.000067,0.000022,0.105469
D_3,-0.067810,0.000381,0.000028,0.000000,0.000000,0.067383
D_4,-0.013786,0.000009,0.000003,0.000003,0.000000,0.013672
D_5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [205]:
B = WW.dod_migration.astype(np.float16)

In [206]:
B

array([[0.95, 0.05, 0.  , 0.  , 0.  , 0.  ],
       [0.9 , 0.05, 0.05, 0.  , 0.  , 0.  ],
       [0.1 , 0.05, 0.05, 0.8 , 0.  , 0.  ],
       [0.05, 0.05, 0.05, 0.05, 0.8 , 0.  ],
       [0.01, 0.01, 0.02, 0.02, 0.04, 0.9 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]], dtype=float16)

In [207]:
B_03 = B @ B @ B
B_03

array([[0.946   , 0.05    , 0.002625, 0.001999, 0.      , 0.      ],
       [0.9077  , 0.05    , 0.004745, 0.005997, 0.03198 , 0.      ],
       [0.2356  , 0.02438 , 0.02155 , 0.05276 , 0.08954 , 0.5757  ],
       [0.12476 , 0.011116, 0.006435, 0.02316 , 0.04965 , 0.7847  ],
       [0.02528 , 0.002497, 0.001577, 0.003881, 0.014946, 0.9517  ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 1.      ]],
      dtype=float16)

In [208]:
B_06 = B @ B @ B @ B @ B @ B
B_06

array([[9.4092e-01, 4.9866e-02, 2.7885e-03, 2.3746e-03, 1.9321e-03,
        3.0785e-03],
       [9.0674e-01, 4.8157e-02, 2.8114e-03, 2.6264e-03, 2.7981e-03,
        3.7872e-02],
       [2.5903e-01, 1.4336e-02, 1.6794e-03, 3.3245e-03, 6.6681e-03,
        7.1436e-01],
       [1.3367e-01, 7.3318e-03, 7.4625e-04, 1.3847e-03, 2.8229e-03,
        8.5400e-01],
       [2.7420e-02, 1.5087e-03, 1.6081e-04, 2.9659e-04, 6.3705e-04,
        9.7021e-01],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00]], dtype=float16)

In [209]:
B_12 = B_06 @ B_06

In [210]:
# A_06_v1 = np.matrix(MM_06_v1).astype(np.float16)
# A_06_v2 = np.matrix(MM_06_v2).astype(np.float16)

In [211]:
A_12_v1 = np.matrix(MM_12_v1).astype(np.float16)
A_12_v2 = np.matrix(MM_12_v2).astype(np.float16)

In [217]:
A = (A_12_v1 - B_12)
print(A)

[[-7.8125e-03 -2.1362e-04  1.7357e-04  3.2043e-04  3.4332e-04  5.4703e-03]
 [-2.9297e-02  9.1553e-05  4.8828e-04  3.0327e-04  3.2425e-04  2.6184e-02]
 [-9.8267e-02 -2.3956e-03 -7.6914e-04 -1.3638e-04 -3.8958e-04  1.0254e-01]
 [-5.6702e-02 -1.8501e-03 -1.1444e-05 -3.4189e-04 -2.8944e-04  5.9082e-02]
 [-1.5282e-02 -7.9203e-04  1.3661e-04  1.4770e-04 -5.9545e-05  1.5625e-02]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]]


In [225]:
A.

matrix([[-7.8125e-03, -2.1362e-04,  1.7357e-04,  3.2043e-04,  3.4332e-04,
          5.4703e-03],
        [-2.9297e-02,  9.1553e-05,  4.8828e-04,  3.0327e-04,  3.2425e-04,
          2.6184e-02],
        [-9.8267e-02, -2.3956e-03, -7.6914e-04, -1.3638e-04, -3.8958e-04,
          1.0254e-01],
        [-5.6702e-02, -1.8501e-03, -1.1444e-05, -3.4189e-04, -2.8944e-04,
          5.9082e-02],
        [-1.5282e-02, -7.9203e-04,  1.3661e-04,  1.4770e-04, -5.9545e-05,
          1.5625e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00]], dtype=float16)

In [233]:
#help(np.linalg.norm)

In [234]:
#help(A.n)

In [212]:
print(sum(abs(A_12_v1 - B_12)).sum())
print(sum(abs(A_12_v2 - B_12)).sum())

0.4258
0.1259


In [213]:
print(abs(A_12_v1 - B_12).max())
print(abs(A_12_v2 - B_12).max())

0.10254
0.02002


In [179]:
print(sum(abs(A_12_v1 - B_12)).sum())
print(sum(abs(A_12_v2 - B_12)).sum())

0.5425
0.1465


In [185]:
print(abs(A_12_v1 - B_12).max())
print(abs(A_12_v2 - B_12).max())

0.1255
0.02686


In [155]:
print(sum(abs(A_06_v1 - B_06)).sum())
print(sum(abs(A_06_v2 - B_06)).sum())

0.2556
0.0723


In [186]:
print(abs(A_06_v1 - B_06).max())
print(abs(A_06_v2 - B_06).max())

0.062
0.01221


In [117]:
print(sum(abs(A_06_v1 - B_06)).sum())
print(sum(abs(A_06_v2 - B_06)).sum())

0.262
0.09174


In [91]:
A_03 = np.matrix(MM_03).astype(np.float16)

In [92]:
A_03

matrix([[0.9497  , 0.046   , 0.002518, 0.001953, 0.      , 0.      ],
        [0.915   , 0.0437  , 0.004288, 0.005604, 0.03137 , 0.      ],
        [0.2449  , 0.0231  , 0.02388 , 0.0546  , 0.0887  , 0.565   ],
        [0.1226  , 0.00922 , 0.00553 , 0.02742 , 0.05093 , 0.784   ],
        [0.02306 , 0.002745, 0.002195, 0.00302 , 0.02031 , 0.9487  ],
        [0.      , 0.      , 0.      , 0.      , 0.      , 1.      ]],
       dtype=float16)

In [93]:
sum(abs(A_03 - B_03)).sum()

0.0727

In [187]:
abs(A_03 - B_03).max()

0.01074

In [55]:
sum(abs(A_03 - B_03)).sum()

0.1111